# PyTorch Tutorial: GETTING STARTED
---

* **Reference**
   * [PyTorch](https://pytorch.org/)
   * [PyTorch Tutorials](https://pytorch.org/tutorials/)


## DEEP LEARNING WITH PYTORCH: [A 60 MINUTE BLITZ](https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html)
---

Goal of this tutorial:


* Understand PyTorch’s Tensor library and neural networks at a high level.
* Train a small neural network to classify images


### [Neural Networks](https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html#sphx-glr-beginner-blitz-neural-networks-tutorial-py)
---

Neural networks can be constructed using the `torch.nn` package.


Now that you had a glimpse of autograd, nn depends on autograd to define models and differentiate them. An nn.Module contains layers, and a method forward(input)that returns the output.


A typical training procedure for a neural network is as follows:

* Define the neural network that has some learnable parameters (or weights)
* Iterate over a dataset of inputs
* Process input through the network
* Compute the loss (how far is the output from being correct)
* Propagate gradients back into the network’s parameters
* Update the weights of the network, typically using a simple update rule: `weight = weight - learning_rate * gradient`

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
print('PyTorch: ', torch.__version__)

PyTorch:  0.4.1


#### Define the Network
---

In [2]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
            
        return num_features

In [3]:
"""
Base class for all neural network modules.

Your models should also subclass this class.
"""
nn.Module()

Module()

In [4]:
type(nn.Module)

type

In [5]:
dir(nn.Module)

['__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_all_buffers',
 '_apply',
 '_get_name',
 '_load_from_state_dict',
 '_slow_forward',
 '_tracing_name',
 '_version',
 'add_module',
 'apply',
 'children',
 'cpu',
 'cuda',
 'double',
 'dump_patches',
 'eval',
 'extra_repr',
 'float',
 'forward',
 'half',
 'load_state_dict',
 'modules',
 'named_children',
 'named_modules',
 'named_parameters',
 'parameters',
 'register_backward_hook',
 'register_buffer',
 'register_forward_hook',
 'register_forward_pre_hook',
 'register_parameter',
 'share_memory',
 'state_dict',
 'to',
 'train',
 'type',
 'zero_grad']

Instantiate a neural network and show the architecture

In [6]:
net = Net()

print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


You just have to define the `forward` function, and the `backward` function (where gradients are computed) is automatically defined for you using `autograd`. You can use any of the Tensor operations in the `forward` function.


```python
"""
Returns an iterator over module parameters.

This is typically passed to an optimizer.
"""
net.parameters()
```


> Learnable parameters

In [7]:
params = list(net.parameters())
print(len(params))
for param in params:
    print(param.size())

10
torch.Size([6, 1, 5, 5])
torch.Size([6])
torch.Size([16, 6, 5, 5])
torch.Size([16])
torch.Size([120, 400])
torch.Size([120])
torch.Size([84, 120])
torch.Size([84])
torch.Size([10, 84])
torch.Size([10])


Let try a random 32x32 input Note: Expected input size to this net(LeNet) is 32x32. To use this net on MNIST dataset, please resize the images from the dataset to 32x32.

In [8]:
input_tensor = torch.randn(1, 1, 32, 32)
out = net(input_tensor)

out

tensor([[ 0.0206,  0.0614,  0.0117,  0.0244,  0.0656,  0.0896,  0.0369,  0.0471,
         -0.0476, -0.0202]], grad_fn=<ThAddmmBackward>)

Zero the gradient buffers of all parameters and backprops with random gradients:

In [9]:
"""
Sets gradients of all model parameters to zero.
"""
net.zero_grad()

"""
Computes the gradient of current tensor w.r.t. graph leaves.

The graph is differentiated using the chain rule.
"""
out.backward(torch.randn(1, 10))

> `torch.nn` only supports **mini-batches**. The entire `torch.nn` package only supports inputs that are a mini-batch of samples, and not a single sample.
>
> For example, `nn.Conv2d` will take in **a 4D Tensor** of `nSamples x nChannels x Height x Width`.
> If you have a single sample, just use `input_tensor.unsqueeze(0)` **to add a fake batch dimension**.

In [10]:
test = torch.ones(1, 32, 32)

test.shape

torch.Size([1, 32, 32])

In [11]:
test.unsqueeze_(0)

test.shape

torch.Size([1, 1, 32, 32])

Before proceeding further, let’s recap all the classes you’ve seen so far.


**Recap**:
* **torch.Tensor** - A multi-dimensional array with support for autograd operations like backward(). Also holds the gradient w.r.t. the tensor.
* **nn.Module** - Neural network module. Convenient way of encapsulating parameters, with helpers for moving them to GPU, exporting, loading, etc.
* **nn.Parameter** - A kind of Tensor, that is automatically registered as a parameter when assigned as an attribute to a Module.
* **autograd.Function** - Implements forward and backward definitions of an autograd operation. Every Tensor operation, creates at least a single Function node, that connects to functions that created a Tensor and encodes its history.


**At this point, we covered**:
* Defining a neural network
* Processing inputs and calling backward


**Still Left**:
* Computing the loss
* Updating the weights of the network

#### Loss Function
---

A loss function **takes the (output, target) pair of inputs**, and computes a value that estimates **how far away the output is from the target**.

In [12]:
output_tensor = net(input_tensor)
target = torch.randn(10)    # a dummy target, for example
target = target.view(1, -1)    # make it the same shape as output

"""
Creates a criterion that measures the mean squared error between
`n` elements in the input `x` and target `y`.
"""
criterion = nn.MSELoss()    # a simple loss function (mean-squared error, MSE)

loss = criterion(output_tensor, target)

print(loss)

tensor(0.4519, grad_fn=<MseLossBackward>)


So, when we call `loss.backward()`, the whole graph is differentiated w.r.t. the loss, and all Tensors in the graph that has `requires_grad=True` will have their `.grad` Tensor accumulated with the gradient.

In [13]:
loss.grad_fn.requires_grad

True


input_tensor -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d  
-> view -> linear -> relu -> linear -> relu -> linear  
-> MSELoss  
-> loss

In [14]:
loss.grad_fn

In [15]:
loss.grad_fn.next_functions[0][0]

In [16]:
loss.grad_fn.next_functions[0][0].next_functions[0][0]

#### Bockprop
---

To backpropagate the error all we have to do is to `loss.backward()`. You need to **clear the existing gradients** though, else gradients will be accumulated to existing gradients.

Now we shall call `loss.backward()`, and have a look at conv1’s bias gradients before and after the backward.

In [17]:
net.zero_grad()    # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward: ', net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward: ', net.conv1.bias.grad)

conv1.bias.grad before backward:  tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward:  tensor([ 0.0014, -0.0065,  0.0031,  0.0147, -0.0087,  0.0113])


#### Update the Weights
---

The simplest update rule used in practice is the Stochastic Gradient Descent (SGD).

In [18]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

#### Other Optimizers
---

However, as you use neural networks, you want to use various different update rules such as SGD, Nesterov-SGD, Adam, RMSProp, etc. To enable this, we built a small package: `torch.optim` that implements all these methods. Using it is very simple:

In [19]:
import torch.optim as optim

dir(optim)

['ASGD',
 'Adadelta',
 'Adagrad',
 'Adam',
 'Adamax',
 'LBFGS',
 'Optimizer',
 'RMSprop',
 'Rprop',
 'SGD',
 'SparseAdam',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'lr_scheduler']

In [20]:
# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()    # zero the gradient buffers
output_tensor = net(input_tensor)
loss = criterion(output_tensor, target)
loss.backward()
# Performs a single optimization step.
optimizer.step()    # Does the update